<img src="https://www.python.org/static/img/python-logo.png" width="300" alt="Python logo"  />
<font color='blue'>
$\Large\text{Pontificia Universidad Católica del Perú}$</font>

$$\Large \textit{Estadística Aplicada}$$

$$\large\textbf{Trabajo Clase 14: Regresión logística}$$

Nombe alumno: Ana Muñoz Maquera



Caso de estudio: **"Base financiera"**

Haciendo uso de la base entregada desarrollar un modelo de regresión logística usando como variable target_mora:


Total de puntos (20 ptos)


In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


import statsmodels.api as sm
from statsmodels.formula.api import ols

### **Paso 1**
1. Carga y muestreo. 
  *   Cargar la base de datos para la actividad.  **(1pto)**
  *   Elegir las muestras de train y test con una proporción de 70% y 30%  **(2ptos)**

In [ ]:
df = pd.read_csv('Base_financiera_1.csv', sep=';', index_col='ID')
df.sample(2)

,Edad,Estado_civil,Genero,Zona_geografica,Ingreso,Numero_entidades_sbs,Tenencia_Saldo_rcc,Tenencia_prestamos_rcc,Tenencia_hipotecario_rcc,Saldo_tc_rcc,...,Antiguedad_pld,Antiguedad_activos,Nro_meses_rcc,Tenencia_ahorros_6m,Tenencia_depo_plazo_6m,Tenencia_cts_6m,Tenencia_FFMM_6m,Nro_meses_visitaOficina_5M,cant_vecesvisitaoficina_5m,Target_mora
ID,,,,,,,,,,,,,,,,,,,,,
6882,55,Soltero,Masculino,LIMA MODERNA,3000,1,1,0,0,0,...,0,0,20,1,1,0,1,5,6,SIN ATRASO
2283,63,Casado,Masculino,LIMA CENTRO,500,3,1,0,0,0,...,0,0,97,1,1,0,1,4,7,SIN ATRASO


In [ ]:
# Elimino zona geografica por la cantidad de variables unicas
df = df.drop(columns='Zona_geografica')
# PASANDO TARGET A 0 Y 1 y todas las variables categoricas
dummies = pd.get_dummies(df[['Target_mora', 'Estado_civil', 'Genero']], drop_first=True)
df = pd.concat([df, dummies], axis=1)
df = df.drop(columns=['Estado_civil', 'Genero', 'Target_mora'], axis=1)

In [ ]:
# Eligiendo la variable target
y = df['Target_mora_SIN ATRASO']
x = df.drop(columns='Target_mora_SIN ATRASO')

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2023, test_size=0.30, shuffle=True)

In [ ]:
# Escalando la data
scaler = StandardScaler()
norm_x = scaler.fit(x_train)

# Dataset despues de la normalizacion
x_train_norm = norm_x.transform(x_train)
x_test_norm = norm_x.transform(x_test)

x_train = pd.DataFrame(x_train_norm, columns=x_train.columns)
x_test = pd.DataFrame(x_test_norm, columns=x_test.columns)

# # normalizando Ingresos
# norm_y = scaler.fit(y_train.values.reshape(-1, 1))
# y_train_norm = norm_y.transform(y_train.values.reshape(-1, 1))
# y_test_norm = norm_y.transform(y_test.values.reshape(-1, 1))

# y_train = pd.DataFrame(y_train_norm)#, columns=y_train.index)
# y_test = pd.DataFrame(y_test_norm)#, #columns=y_test.index)

### **Paso 2**
2. Seleccionar un grupo de variables que sean adecuadas para el modelo. Describa los criterios o el método de selección utilizado. Puede ser explorando las caracteristicas de cada variable o creando una primera versión del modelo **(5 ptos)**




In [ ]:
# Evaluo correlaciones
corr = x_train.corr()

# Obteniendo correlaciones mayores a 0.9 y -0.9
corr = corr.where(np.tril(np.ones(corr.shape)).astype(np.bool))
corr = corr.unstack().dropna().sort_values(ascending=False)
corr = corr[corr!=1]

corre_high = corr[corr>=0.9]
corre_low = corr[corr<=-0.9]

# variables con alta correlacion
corre_high 
# Nro_productos_activo_rcc  y Cantidad_productos_rcc varibales con 1 de correlacion
#corre_low # vacio

<ipython-input-137-33d3271218d7>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  corr = corr.where(np.tril(np.ones(corr.shape)).astype(np.bool))


Numero_entidades_sbs  Nro_bancos_tc_saldo    0.918498
dtype: float64

In [ ]:
# Drop variables altamente correlacionadas
x_train = x_train.drop(columns=['Nro_bancos_tc_saldo','Cantidad_productos_rcc'], axis=1)

# Columnas que quedan
print('Columas que quedan :', len(x_test.columns))

Columas que quedan : 49


In [ ]:
# ## PROBANDO CON FEATURE SELECTION
# from sklearn.feature_selection import SequentialFeatureSelector

# selector = SequentialFeatureSelector(
#     estimator=LogisticRegression(solver='lbfgs', max_iter=100), n_features_to_select=10,
#     direction='backward', cv=3, scoring='f1'
# )
# selector.fit(x_train,y_train)

# selected_forward = x_train.columns[selector.get_support()]
# columnas = selected_forward.sort_values().tolist()
# columnas

### **Paso 3**
*   Entrenar el modelo en la subpoblación de train **(3ptos)**
*   Analizar la significancia de las variables en el modelo elegido **(3ptos)**
*   Calcular e interpretar la métrica de bondad de ajuste para el modelo "F1_score" en los subgrupos de train y test (comparar) **(3ptos)**:


In [ ]:
# Drop variables altamente correlacionadas
x_test = x_test.drop(columns=['Nro_bancos_tc_saldo','Cantidad_productos_rcc'], axis=1)
x_test.shape

(3193, 47)

In [ ]:
# Modelo
model_lr = LogisticRegression(max_iter=100)
model_lr.fit(x_train, y_train)
y_pred = model_lr.predict(x_test)

coefficients = model_lr.coef_.ravel()
pd.DataFrame({'Variables': x_train.columns, 'Coeficiente': coefficients}).round(4)

,Variables,Coeficiente
0,Edad,0.2079
1,Ingreso,0.1247
2,Numero_entidades_sbs,-0.3260
3,Tenencia_Saldo_rcc,-0.3055
4,Tenencia_prestamos_rcc,-0.0571
5,Tenencia_hipotecario_rcc,0.1949
6,Saldo_tc_rcc,-2.0896
7,Saldo_prestamo_rcc,-0.7667
8,Saldo_hipotecario_rcc,-1.5563
9,deuda_rcc,-1.4890


In [ ]:
from sklearn.metrics import classification_report
#################################
# MORA SIN ATRASO = 1 #
y_pred_train = model_lr.predict(x_train)
y_pred_test = model_lr.predict(x_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))#


              precision    recall  f1-score   support

           0       0.98      0.33      0.49       574
           1       0.95      1.00      0.97      6876

    accuracy                           0.95      7450
   macro avg       0.97      0.66      0.73      7450
weighted avg       0.95      0.95      0.94      7450

              precision    recall  f1-score   support

           0       0.93      0.32      0.48       244
           1       0.95      1.00      0.97      2949

    accuracy                           0.95      3193
   macro avg       0.94      0.66      0.73      3193
weighted avg       0.95      0.95      0.93      3193



In [ ]:
f1_score(y_train,  y_pred_train)
f1_score(y_test, y_pred_test)

0.9717682020802378

### **Paso 4**

Encontrar el punto de corte óptimo e interpretar:  **(3ptos)**





In [ ]:
from sklearn.metrics import  f1_score, roc_curve, precision_recall_curve

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
class LogisticRegressionWithThreshold(LogisticRegression):
    def predict(self, X, threshold=None):
        if threshold == None: # If no threshold passed in, simply call the base class predict, effectively threshold=0.5
            return LogisticRegression.predict(self, X)
        else:
            y_scores = LogisticRegression.predict_proba(self, X)[:, 1]
            y_pred_with_threshold = (y_scores >= threshold).astype(int)

            return y_pred_with_threshold
    
    def threshold_from_optimal_f_score(self, X, y):
        y_scores = LogisticRegression.predict_proba(self, X)[:, 1]
        precisions, recalls, thresholds = precision_recall_curve(y, y_scores)

        fscores = (2 * precisions * recalls) / (precisions + recalls)
        
        optimal_idx = np.argmax(fscores)
        
        return thresholds[optimal_idx], fscores[optimal_idx]

In [ ]:
lrt = LogisticRegressionWithThreshold()
lrt.fit(x_train, y_train)

threshold, optimal_f1_score = lrt.threshold_from_optimal_f_score(x_train, y_train)
y_pred = lrt.predict(x_train, threshold)

print('punto de corte optimo', threshold)
print('F score optimo', optimal_f1_score)


punto de corte optimo 0.4922523791866824
F score optimo 0.9724835537950061


* El punto de corte optimo seria de 0.49